In [1]:
import torch
import torch.nn as nn
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os


In [2]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

Using device: mps


In [10]:
class TestDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.folder = folder
        self.transform = transform
        self.images = [f for f in os.listdir(folder) if f.endswith((".png", ".jpg", ".jpeg"))]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        fname = self.images[idx]
        path = os.path.join(self.folder, fname)

        img = Image.open(path).convert("RGB")
        if self.transform:
            img = self.transform(img)

    # ✅ Label is simply the first character of filename (e.g. "a_test.jpg")
        label = ord(fname[0].lower()) - ord("a")

        return img, label, fname

In [11]:
test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

test_dir = "data/test"
test_dataset = TestDataset(test_dir, transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Total test images:", len(test_dataset))

Total test images: 28


In [15]:
from torchvision.models import resnet18, ResNet18_Weights

# Must match training setup
model = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

# Replace final layer with 26 classes
model.fc = nn.Linear(model.fc.in_features, 26)

# Load trained weights
model.load_state_dict(torch.load("../models/asl_model.pth", map_location=device))
model.to(device)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [13]:
correct, total = 0, 0
all_preds, all_labels, all_files = [], [], []

with torch.no_grad():
    for images, labels, fnames in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_files.extend(fnames)

accuracy = correct / total
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.8571


In [14]:
idx_to_class = {i: chr(i + ord("a")) for i in range(26)}

for i in range(10):  # show first 10
    print(f"File: {all_files[i]} | True: {idx_to_class[all_labels[i]]} | Pred: {idx_to_class[all_preds[i]]}")

File: F_test.jpg | True: f | Pred: f
File: G_test.jpg | True: g | Pred: g
File: L_test.jpg | True: l | Pred: l
File: M_test.jpg | True: m | Pred: m
File: R_test.jpg | True: r | Pred: r
File: S_test.jpg | True: s | Pred: s
File: X_test.jpg | True: x | Pred: x
File: Y_test.jpg | True: y | Pred: y
File: U_test.jpg | True: u | Pred: u
File: T_test.jpg | True: t | Pred: t
